

# Logistic Regression + Grid Search

In [ ]:
# !pip install imblearn

In [ ]:
# !pip install -U threadpoolctl

In [ ]:
# !pip install imbalanced-learn

In [ ]:
# !pip install scikit-learn --user

In [6]:
import gc
import time
import numpy as np
import pandas as pd
from datetime import datetime

### Model training and hyperparameter tuning

In [7]:
from sklearn.metrics import mean_squared_error, accuracy_score, f1_score, roc_auc_score
from sklearn.model_selection import GridSearchCV

Use Data after Part 2 Feature Selection (Selected using Random Forest)

In [8]:
train = pd.read_csv("../Datasets/Final Datasets/Actual Dataset/train.csv")
test = pd.read_csv("../Datasets/Final Datasets/Actual Dataset/test.csv")
train_labels = pd.read_csv("../Datasets/Final Datasets/Actual Dataset/train_labels.csv")
test_labels = pd.read_csv("../Datasets/Final Datasets/Actual Dataset/test_labels.csv")

In [9]:
train.shape, train_labels.shape

((783, 30), (783, 1))

In [10]:
train

,CoefficientVariationofStockPrice,PercentageChangefromprevreportedEPS,PriceChange,QuestionsSentiment,ConferenceSentiment,Has_QnA,Reddit_Sentiment,RedditSD,TwitterSentiment,TwitterSD,...,Energy,Financial Services,Healthcare,Industrials,Real Estate,Technology,quarter_1,quarter_2,quarter_3,quarter_4
0,0.307267,-0.557576,1.409556,-0.714286,0.578947,1,0.068178,0.423146,0.219354,0.441965,...,0,0,0,0,0,0,1.0,0.0,0.0,0.0
1,0.242076,0.306452,1.378303,-0.666667,0.466667,1,0.062141,0.433917,0.082585,0.451483,...,1,0,0,0,0,0,1.0,0.0,0.0,0.0
2,0.043960,0.057471,-0.009751,-0.750000,0.652174,1,0.000000,0.000000,0.415066,0.440203,...,0,0,0,0,0,0,1.0,0.0,0.0,0.0
3,0.106001,0.000000,-0.025346,-0.714286,0.566667,1,0.219251,0.441184,0.145072,0.412363,...,0,0,0,0,0,1,0.0,0.0,0.0,1.0
4,0.036320,-0.256881,-0.100652,-0.272727,0.761905,1,0.203557,0.318592,0.213863,0.434037,...,0,0,0,0,0,1,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
778,0.078014,-0.906250,0.114644,-0.333333,0.555556,1,0.089879,0.420355,0.032813,0.309138,...,0,0,0,0,0,0,0.0,1.0,0.0,0.0
779,0.105664,0.105263,-0.402886,-0.272727,0.696970,1,0.141945,0.422794,0.138581,0.485417,...,0,0,0,0,0,1,0.0,0.0,1.0,0.0
780,0.218293,-0.257143,0.484055,0.200000,0.673469,1,0.345041,0.405775,0.127045,0.399143,...,0,0,0,0,0,1,1.0,0.0,0.0,0.0
781,0.096284,-0.536232,0.192951,-0.500000,0.777778,1,0.000000,0.177206,0.264362,0.402587,...,0,0,0,0,0,0,1.0,0.0,0.0,0.0


In [11]:
train_labels

,EPSBeats
0,0
1,1
2,1
3,0
4,1
...,...
778,0
779,1
780,1
781,1


In [12]:
test.shape, test_labels.shape

((196, 30), (196, 1))

In [13]:
train.columns

Index(['CoefficientVariationofStockPrice',
       'PercentageChangefromprevreportedEPS', 'PriceChange',
       'QuestionsSentiment', 'ConferenceSentiment', 'Has_QnA',
       'Reddit_Sentiment', 'RedditSD', 'TwitterSentiment', 'TwitterSD',
       'SocialMediaCovariance', 'TwitterSkew', 'TwitterKurtosis', 'RedditSkew',
       'RedditKurtosis', 'EPSEstimate', 'Has_Reddit', 'Communication Services',
       'Consumer Cyclical', 'Consumer Defensive', 'Energy',
       'Financial Services', 'Healthcare', 'Industrials', 'Real Estate',
       'Technology', 'quarter_1', 'quarter_2', 'quarter_3', 'quarter_4'],
      dtype='object')

In [14]:
test.columns 

Index(['CoefficientVariationofStockPrice',
       'PercentageChangefromprevreportedEPS', 'PriceChange',
       'QuestionsSentiment', 'ConferenceSentiment', 'Has_QnA',
       'Reddit_Sentiment', 'RedditSD', 'TwitterSentiment', 'TwitterSD',
       'SocialMediaCovariance', 'TwitterSkew', 'TwitterKurtosis', 'RedditSkew',
       'RedditKurtosis', 'EPSEstimate', 'Has_Reddit', 'Communication Services',
       'Consumer Cyclical', 'Consumer Defensive', 'Energy',
       'Financial Services', 'Healthcare', 'Industrials', 'Real Estate',
       'Technology', 'quarter_1', 'quarter_2', 'quarter_3', 'quarter_4'],
      dtype='object')

In [15]:
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE 
from imblearn.under_sampling import EditedNearestNeighbours
from imblearn.combine import SMOTEENN

In [16]:
from sklearn.linear_model import LogisticRegression

In [17]:
smote = SMOTE(random_state = 1, sampling_strategy=1.0)
enn = EditedNearestNeighbours(n_neighbors=5, sampling_strategy = 'majority')

In [18]:
model_lr = Pipeline([('smt', smote),('enn', enn),('LR',LogisticRegression(max_iter = 300))])

In [19]:
model_lr_without_balancing = Pipeline([('LR',LogisticRegression(max_iter = 300))])

In [20]:
train.columns

Index(['CoefficientVariationofStockPrice',
       'PercentageChangefromprevreportedEPS', 'PriceChange',
       'QuestionsSentiment', 'ConferenceSentiment', 'Has_QnA',
       'Reddit_Sentiment', 'RedditSD', 'TwitterSentiment', 'TwitterSD',
       'SocialMediaCovariance', 'TwitterSkew', 'TwitterKurtosis', 'RedditSkew',
       'RedditKurtosis', 'EPSEstimate', 'Has_Reddit', 'Communication Services',
       'Consumer Cyclical', 'Consumer Defensive', 'Energy',
       'Financial Services', 'Healthcare', 'Industrials', 'Real Estate',
       'Technology', 'quarter_1', 'quarter_2', 'quarter_3', 'quarter_4'],
      dtype='object')

In [21]:
def param_grid_search(train, model, train_labels = train_labels):
    # Step 1.Generate Parameter Space
    print('param_grid_search')
    features = train.columns.tolist() #Features
    
    # add more possible parameters and values
    parameter_space = [
    {
        "LR__C"       : [100, 10, 1, 0.1, 0.01],
        "LR__solver"  : ["newton-cg", "lbfgs"],
        "LR__penalty" : ["l2"],
    },
    {
        "LR__C"       : [100, 10, 1, 0.1, 0.01],
        "LR__solver"  : ["liblinear"],
        "LR__penalty" : ["l1", "l2"],
    }
]
    
    # Step 2.Tune Hyper Parameters
    print("Tuning hyper-parameters for mse")
    grid = GridSearchCV(model, parameter_space, cv=10, scoring="accuracy",refit = True)
    print(train[features].shape, train_labels.shape)
    grid.fit(train[features].values, train_labels.values)
    
    # Step 3.Generate Best parameters
    print("best_params_:")
    print(grid.best_params_)
    means = grid.cv_results_["mean_test_score"]
    stds = grid.cv_results_["std_test_score"]
    for mean, std, params in zip(means, stds, grid.cv_results_["params"]):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    return grid

In [ ]:
grid = param_grid_search(train, model = model_lr)

param_grid_search
Tuning hyper-parameters for mse
(783, 30) (783, 1)


C:\Users\User\AppData\Roaming\Python\Python38\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\User\AppData\Roaming\Python\Python38\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.

best_params_:
{'LR__C': 0.1, 'LR__penalty': 'l1', 'LR__solver': 'liblinear'}
0.697 (+/-0.096) for {'LR__C': 100, 'LR__penalty': 'l2', 'LR__solver': 'newton-cg'}
0.697 (+/-0.096) for {'LR__C': 100, 'LR__penalty': 'l2', 'LR__solver': 'lbfgs'}
0.693 (+/-0.089) for {'LR__C': 10, 'LR__penalty': 'l2', 'LR__solver': 'newton-cg'}
0.693 (+/-0.089) for {'LR__C': 10, 'LR__penalty': 'l2', 'LR__solver': 'lbfgs'}
0.695 (+/-0.106) for {'LR__C': 1, 'LR__penalty': 'l2', 'LR__solver': 'newton-cg'}
0.695 (+/-0.106) for {'LR__C': 1, 'LR__penalty': 'l2', 'LR__solver': 'lbfgs'}
0.697 (+/-0.083) for {'LR__C': 0.1, 'LR__penalty': 'l2', 'LR__solver': 'newton-cg'}
0.697 (+/-0.083) for {'LR__C': 0.1, 'LR__penalty': 'l2', 'LR__solver': 'lbfgs'}
0.686 (+/-0.060) for {'LR__C': 0.01, 'LR__penalty': 'l2', 'LR__solver': 'newton-cg'}
0.686 (+/-0.060) for {'LR__C': 0.01, 'LR__penalty': 'l2', 'LR__solver': 'lbfgs'}
0.697 (+/-0.096) for {'LR__C': 100, 'LR__penalty': 'l1', 'LR__solver': 'liblinear'}
0.697 (+/-0.096) for {'

In [ ]:
grid_without_balance = param_grid_search(train, model = model_lr_without_balancing)

param_grid_search
Tuning hyper-parameters for mse
(783, 30) (783, 1)


C:\Users\User\AppData\Roaming\Python\Python38\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\User\AppData\Roaming\Python\Python38\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\User\AppData\Roaming\Python\Python38\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\User\AppData\Roaming\Python\Python38\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the s

C:\Users\User\AppData\Roaming\Python\Python38\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\User\AppData\Roaming\Python\Python38\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\User\AppData\Roaming\Python\Python38\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\User\AppData\Roaming\Python\Python38\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the s

C:\Users\User\AppData\Roaming\Python\Python38\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\User\AppData\Roaming\Python\Python38\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\User\AppData\Roaming\Python\Python38\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\User\AppData\Roaming\Python\Python38\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the s

C:\Users\User\AppData\Roaming\Python\Python38\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\User\AppData\Roaming\Python\Python38\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\User\AppData\Roaming\Python\Python38\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\User\AppData\Roaming\Python\Python38\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the s

C:\Users\User\AppData\Roaming\Python\Python38\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\User\AppData\Roaming\Python\Python38\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\User\AppData\Roaming\Python\Python38\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\User\AppData\Roaming\Python\Python38\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the s

best_params_:
{'LR__C': 10, 'LR__penalty': 'l1', 'LR__solver': 'liblinear'}
0.702 (+/-0.088) for {'LR__C': 100, 'LR__penalty': 'l2', 'LR__solver': 'newton-cg'}
0.702 (+/-0.088) for {'LR__C': 100, 'LR__penalty': 'l2', 'LR__solver': 'lbfgs'}
0.708 (+/-0.086) for {'LR__C': 10, 'LR__penalty': 'l2', 'LR__solver': 'newton-cg'}
0.708 (+/-0.086) for {'LR__C': 10, 'LR__penalty': 'l2', 'LR__solver': 'lbfgs'}
0.706 (+/-0.077) for {'LR__C': 1, 'LR__penalty': 'l2', 'LR__solver': 'newton-cg'}
0.706 (+/-0.077) for {'LR__C': 1, 'LR__penalty': 'l2', 'LR__solver': 'lbfgs'}
0.693 (+/-0.082) for {'LR__C': 0.1, 'LR__penalty': 'l2', 'LR__solver': 'newton-cg'}
0.693 (+/-0.082) for {'LR__C': 0.1, 'LR__penalty': 'l2', 'LR__solver': 'lbfgs'}
0.682 (+/-0.036) for {'LR__C': 0.01, 'LR__penalty': 'l2', 'LR__solver': 'newton-cg'}
0.682 (+/-0.036) for {'LR__C': 0.01, 'LR__penalty': 'l2', 'LR__solver': 'lbfgs'}
0.705 (+/-0.091) for {'LR__C': 100, 'LR__penalty': 'l1', 'LR__solver': 'liblinear'}
0.702 (+/-0.088) for {'L

C:\Users\User\AppData\Roaming\Python\Python38\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [ ]:
grid_without_balance.best_score_

0.7088607594936709

In [ ]:
# Train score for accuracy, f1, roc_auc 
predictions_train = grid_without_balance.best_estimator_.predict(train)
print(accuracy_score(train_labels, predictions_train))
print(roc_auc_score(train_labels, predictions_train))
print(f1_score(train_labels, predictions_train))

0.7305236270753512
0.6450757575757575
0.8166811468288446


In [ ]:
# Test score for accuracy, f1, roc_auc 
predictions = grid_without_balance.best_estimator_.predict(test)
print(accuracy_score(test_labels, predictions))
print(roc_auc_score(test_labels, predictions))
print(f1_score(test_labels, predictions))

0.7653061224489796
0.6768465909090908
0.8424657534246575


In [ ]:
lr_pred_without_balance = pd.DataFrame(predictions, columns = ['predictions'])

In [ ]:
# lr_pred_without_balance.to_csv('model_lr_predictions.csv', index = False)